In [1]:
# @name graph_v3.2_v20190616
# @description notebook to build the NGLY1 Deficiency review knowledge graph v3.2
# @author Núria Queralt Rosinach
# @date 16 June 2019

# Description

This is the notebook for the creation of the first review network and derived hypotheses. 

* Using intermediary variables from workflow objects. In this workflow variables are directly used for the next step. 


* Review network: From Monarch knowledge graph, we built a network seeded by 8 nodes, retrieving their explicit relationships and all the relationships among all these nodes. Seed nodes:

    - 'MONDO:0014109', # NGLY1 deficiency
    - 'HGNC:17646', # NGLY1 human gene
    - 'HGNC:633', # AQP1 human gene
    - 'MGI:103201', # AQP1 mouse gene
    - 'HGNC:7781', # NFE2L1 human gene
    - 'HGNC:24622', # ENGASE human gene
    - 'HGNC:636', # AQP3 human gene
    - 'HGNC:19940' # AQP11 human gene
    

* Connecting paths: query templates.

In [1]:
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary, utils

KeyboardInterrupt: 

## Edges library
### Review edges to integrate into the knowledge graph and prepare them as individual networks

#### TRANSCRIPTOMICS NETWORK
#### import transcriptomics
We retrieved edges from RNA-seq transcriptomics profiles using the `transcriptomics` module:

    - Experimental data sets: from Chow et al. paper [pmid:29346549] (NGLY1 deficiency model on fruit fly)

In [2]:
%%time
# prepare data to graph schema
csv_path = './transcriptomics/ngly1-fly-chow-2018/data/supp_table_1.csv'
data = transcriptomics.read_data(csv_path)
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes = transcriptomics.build_nodes(rna_network)


The function "read_data()" is running...

* This is the size of the raw expression data structure: (15370, 9)
* These are the expression attributes: Index(['FlyBase ID', 'Symbol', 'baseMean', 'log2FoldChange', 'Unnamed: 4',
       'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
    FlyBase ID  Symbol    baseMean  log2FoldChange  Unnamed: 4     lfcSE  \
0  FBgn0030880  CG6788  175.577087       -4.209283     0.05406  0.190308   

        stat         pvalue           padj  
0 -22.118249  2.110000e-108  2.860000e-104  

The raw data is saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/ngly1-fly-chow-2018/data/supp_table_1.csv


Finished read_data().


The function "clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5% ...

* This is the size of the clean expression data structure: (386, 6)
* These are the clean expression attributes: Index(['FlyBase ID', 'Symbol', 'log2FoldChange', 

- Transcriptomics network is returned as both digital object (`rna_edges`, `rna_nodes`) and CSV files at _**graph/**_ (`rna_edges_version.csv`, `rna_nodes_version.csv`)

In [3]:
# print type of objects
print('type edges:', type(rna_edges))
print('type nodes:', type(rna_nodes))
print()

# print objects sizes
print('len edges:', len(rna_edges))
print('len nodes:', len(rna_nodes))
print()

# print object attribute
print('attribute edges:', rna_edges[0].keys())
print('attribute nodes:', rna_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 386
len nodes: 386

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### REGULATION NETWORK
#### import regulation

We retrieved human TF gene expression regulation edges from several sources using the `regulation` module.

In [4]:
%%time
# prepare msigdb data
gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)


The function "prepare_msigdb_data()" is running...

* Number of Transcription Factor Targets (TFT) gene sets: 615

The MSigDB raw network is saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/regulation/msigdb/out/tf_genelist_entrez_msigdb.json. Other reporting files are also saved at the same directory.


Finished prepare_msigdb_data().


The function "load_tf_gene_edges()" is running...

Finished load_tf_gene_edges().


The function "get_gene_id_normalization_dictionaries()" is running...

* Querying BioThings to map gene symbols to HGNC and Entrez IDs...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
48 input query terms found no hit:
	['CACCCBINDINGFACTOR', 'ALPHACP1', 'MEIS1BHOXA9', 'NKX62', 'E2F4DP2', 'CDPCR3HD', 'AP1FJ', 'HMEF2', 
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Saving not found gene symbols at: /home/karolis/Structured revi

- Regulation network is returned as both digital object (`reg_edges`, `reg_nodes`) and CSV files at _**graph/**_ (`regulation_edges_version.csv`, `regulation_nodes_version.csv`)

In [5]:
# print type of objects
print('type edges:', type(reg_edges))
print('type nodes:', type(reg_nodes))
print()

# print objects sizes
print('len edges:', len(reg_edges))
print('len nodes:', len(reg_nodes))
print()

# print object attribute
print('attribute edges:', reg_edges[0].keys())
print('attribute nodes:', reg_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 197267
len nodes: 16966

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### CURATED NETWORK
#### import curation

We retrieved and prepared curated edges using the `curation` module. 

In [6]:
%%time
# graph v3.2
# read network from drive and concat all curated statements
curation_edges, curation_nodes = curation.read_network(version='v20180118')

# prepare data edges and nodes
data_edges = curation.prepare_data_edges(curation_edges)
data_nodes = curation.prepare_data_nodes(curation_nodes)

# prepare curated edges and nodes
curated_network = curation.prepare_curated_edges(data_edges)
curated_concepts = curation.prepare_curated_nodes(data_nodes)


# build edges and nodes files
curation_edges = curation.build_edges(curated_network)
curation_nodes = curation.build_nodes(curated_concepts)


The function "read_network()" is running...

Reading and concatenating all curated statements in the network...

* Curation edge files concatenated shape: (322, 22)

Reading and concatenating all curated nodes in the network...

* Curation node files concatenated shape: (318, 9)

Finished read_network().


The function "prepare_data_edges()" is running...

Preparing curated network...

Saving curated network at curation/...

*Curated edges data structure shape: (321, 9)
*Curated edges data structure fields: Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')

The curated edges are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/curation/curated_edges_v2020-09-10.csv


Finished prepare_data_edges().


The function "prepare_data_nodes()" is running...

Preparing curated nodes...

Saving curated n

- Curated network is returned as both digital object (`curation_edges`, `curation_nodes`) and CSV files at _**graph/**_ (`curated_graph_edges_version.csv`, `curated_graph_nodes_version.csv`)
- The original curated network, i.e. without graph data model normalization, is saved as CSV files at _**curation/**_ (`curated_edges_version.csv`, `curated_nodes_version.csv`)

In [7]:
# print type of objects
print('type edges:', type(curation_edges))
print('type nodes:', type(curation_nodes))
print()

# print objects sizes
print('len edges:', len(curation_edges))
print('len nodes:', len(curation_nodes))
print()

# print object attribute
print('attribute edges:', curation_edges[0].keys())
print('attribute nodes:', curation_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 362
len nodes: 302

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date', 'g2p_mark'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### MONARCH NETWORK
#### import monarch
We retrieved edges from Monarch using the `monarch` module.

Tasks:

- From 8 seed nodes we retrieved 1st shell nodes
- From all seed and 1st shell nodes we retrieved ortho-phenotypes
- We retrieved extra edges among all of them, i.e. extra connectivity between: seed, 1st shell, ortholog-phenotype nodes

In [2]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0014109', # NGLY1 deficiency
    'HGNC:17646', # NGLY1 human gene
    'HGNC:633', # AQP1 human gene
    'MGI:103201', # AQP1 mouse gene
    'HGNC:7781', # NFE2L1 human gene
    'HGNC:24622', # ENGASE human gene
    'HGNC:636', # AQP3 human gene
    'HGNC:19940' # AQP11 human gene
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))

# introduce animal model ortho-phenotypes for seed and 1st shell neighbors
## For seed nodes:
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
## For 1st shell nodes:
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

# network nodes: seed + 1shell + ortholog-phentoype
geneList = sum([seedList,
                neighboursList,
                seed_orthophenoList,
                neighbours_orthophenoList
               ], 
               [])
print('genelist: ',len(geneList))

# get Monarch network
monarch_network = monarch.extract_edges(geneList)
print('network: ',len(monarch_network))

# save edges
monarch.print_network(monarch_network, 'monarch_connections')

# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)


The function "get_neighbours_list()" is running. Its runtime may take some minutes. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 8/8 [00:47<00:00,  5.20s/it]



Finished get_neighbours_list().

916
genelist:  924

The function "extract_edges()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the edges retrieved and you should start over the execution of this function.


 56%|█████▋    | 522/924 [46:07<31:51,  4.76s/it]  

error: <class 'requests.exceptions.SSLError'>
FlyBase:FBal0244826


100%|██████████| 924/924 [1:45:44<00:00,  7.12s/it]    



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

network:  10980

Saving Monarch edges at: '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_v2020-09-09.csv'...


The function "build_edges()" is running...
Detected a new reference database in Monarch not yet implemented in this module. The new source should be added to the dictionary of databases.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'dbPrefixes_dct' dictionary with 'PomBase'
The edge that includes this new reference database is Pandas(Index=7678, object_id='GO:0000224', object_label='peptide-N4-(N-acetyl-beta-glucosaminyl)asparagine amidase activity', reference_id_list='PomBase', relation_id=None, relation_label=None, subject_id='PomBase:SPBC1709.14', subject_label='ngl1')
The method will continue to run without problem, writing the CURIE instead of the URI,u

In [8]:
monarch_network = monarch.read_connections("monarch_connections_v2020-09-04.csv")


* This is the size of the data structure: (117106, 7)
* These are the attributes: Index(['object_id', 'object_label', 'reference_id_list', 'relation_id',
       'relation_label', 'subject_id', 'subject_label'],
      dtype='object')
* This is the first record:
                object_id object_label reference_id_list    relation_id  \
0  Xenbase:XB-GENE-978429          gns               NaN  RO:HOM0000020   

                          relation_label          subject_id subject_label  
0  in 1 to 1 orthology relationship with  NCBIGene:100034131           GNS  


In [9]:
monarch_edges = monarch.build_edges(monarch_network)



The function "build_edges()" is running...
Detected a new reference database in Monarch not yet implemented in this module. The new source should be added to the dictionary of databases.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'dbPrefixes_dct' dictionary with 'PomBase'
The edge that includes this new reference database is Pandas(Index=17628, object_id='GO:0006516', object_label='glycoprotein catabolic process', reference_id_list='PomBase', relation_id='None', relation_label='None', subject_id='PomBase:SPBC1709.14', subject_label='ngl1')
The method will continue to run without problem, writing the CURIE instead of the URI,until the dictionary is updated.
Detected a new reference database in Monarch not yet implemented in this module. The new source should be added to the dictionary of databases.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'dbPrefixes_d

In [10]:
monarch_nodes = monarch.build_nodes(monarch_network)


The function "build_nodes()" is running...
Number of concepts: 9060
Number of nodes CURIEs: 32
List of nodes CURIEs: dict_keys(['NCBIGene', 'Xenbase', 'HGNC', 'HP', 'PomBase', 'ENSEMBL', 'FlyBase', 'MGI', 'ZFIN', 'WormBase', 'SGD', 'RGD', 'dictyBase', 'ZP', 'MONARCH', 'UBERON', 'GO', 'MP', 'FBcv', 'APO', 'WBPhenotype', 'REACT', 'MONDO', 'EcoGene', 'EnsemblGenome', 'CL', 'EFO', 'OMIM', 'ClinVarVariant', 'Coriell', 'MMRRC', 'dbSNP'])

Adding BioThings annotation: gene name, synonyms, description...
symbols: 4463
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-4463...done.
Finished.
597 input query terms found dup hits:
	[('GNS', 10), ('gns', 2), ('AQP2', 8), ('UBXN2B', 11), ('GPR152', 7), ('AQP4', 11), ('Ssmem1', 2), (
664 input query terms found no hit:
	['ENSEMBL:ENSACAG00000021023', 'FlyBase:CG4019', 'ENSEMBL:ENSOANG00000015032', 'ENSEMBL:ENSACAG00000
Pass "returnall=True" to return complete lists of duplicate or 

In [ ]:
# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 117106
len nodes: 9060

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


- Monarch network is returned as both digital object (`monarch_edges`, `monarch_nodes`) and CSV files at _**monarch/**_ (`monarch_edges_version.csv`, `monarch_nodes_version.csv`)

## Graph library
### Create the review knowledge graph
#### import graph

Tasks:

* Load Networks and calculate graph nodes
* Retrieve extra connectivity for the graph from Monarch
* Build the review graph

In [ ]:
%%time
# load networks and calculate graph nodes
graph_nodes_list, reg_graph_edges = graph.graph_nodes(
    curation=curation_edges,
    monarch=monarch_edges,
    transcriptomics=rna_edges,
    regulation=reg_edges
)

# Monarch graph connectivity
## get Monarch edges
monarch_network_graph = monarch.extract_edges(graph_nodes_list)
print('network: ',len(monarch_network_graph))

## save Monarch network
monarch.print_network(monarch_network_graph, 'monarch_connections_graph')

## build Monarch network with graph schema
monarch_graph_edges = monarch.build_edges(monarch_network_graph)
monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

# build review graph
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges,
    transcriptomics=rna_edges,
    regulation=reg_graph_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)


The function "graph_nodes()" is running...

Preparing networks...
Curated:
(362, 10)
Index(['g2p_mark', 'object_id', 'property_description', 'property_id',
       'property_label', 'property_uri', 'reference_date',
       'reference_supporting_text', 'reference_uri', 'subject_id'],
      dtype='object')
Monarch:
(117106, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Transcriptomics:
(386, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')

  7%|▋         | 1132/15195 [1:23:10<21:04:50,  5.40s/it]

- Regulation edges _merged_ with the graph is returned as both digital object (`reg_graph_edges`) and CSV file at _**graph/**_ (`regulation_graph_edges_version.csv`)
- Monarch network is returned as both digital object (`monarch_graph_edges`, `monarch_graph_nodes`) and CSV files at _**monarch/**_ (`monarch_edges_version.csv`, `monarch_nodes_version.csv`) overwritten the previous one.
- Review knowledge graph is returned as both digital object (`edges`, `nodes`) and CSV files at _**graph/**_ (`graph_edges_version.csv`, `graph_nodes_version.csv`)

In [ ]:
# print type of objects
print('type edges:', type(edges))
print('type nodes:', type(nodes))
print()

# print objects sizes
print('len edges:', len(edges))
print('len nodes:', len(nodes))
print()

# print object attribute
print('attribute edges:', edges.columns)
print('attribute nodes:', nodes.columns)

## Neo4jlib library
### Import the graph into Neo4j graph database
#### import neo4jlib

Tasks:

- Create Neo4j server instance
- Import review graph into the Neo4j graph database

In [ ]:
%%time
# create a Neo4j server instance
neo4j_dir = neo4jlib.create_neo4j_instance()
print('The name of the neo4j directory is {}'.format(neo4j_dir))

# import to graph database
## prepare the graph to neo4j format
edges_df = utils.get_dataframe(edges)
nodes_df = utils.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## save files into neo4j import dir
neo4j_path = './{}'.format(neo4j_dir)
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

## import graph into neo4j database
neo4jlib.do_import(neo4j_path)

In [ ]:
# print type of objects
print('type edges:', type(statements))
print('type nodes:', type(concepts))
print()

# print objects sizes
print('len edges:', len(statements))
print('len nodes:', len(concepts))
print()

# print object attribute
print('attribute edges:', statements.columns)
print('attribute nodes:', concepts.columns)

## hypothesis-generation library
### Query the graph for mechanistic explanation, then summarize the extracted paths
#### import hypothesis, summary


Tasks:

* Retrieve orthopheno paths with the `query` method.
* Retrieve orthopheno paths using relaxing node degree parameters with the `query` method.
* Retrieve orthopheno paths from a more open query topology with the `open_query` method.
* Get hypothesis summaries

### Ortopheno query with general nodes/relations removed

In [ ]:
%%time
# get orthopheno paths
seed = list([
        'HGNC:17646',  # NGLY1 human gene
        'HGNC:633'  # AQP1 human gene
])
hypothesis.query(seed,queryname='ngly1_aqp1',port='7687') 

In [ ]:
%%time
# get orthopheno paths relaxing pathway and phenotype node degrees
seed = list([
        'HGNC:17646',  # NGLY1 human gene
        'HGNC:633'  # AQP1 human gene
])
hypothesis.query(seed, queryname='ngly1_aqp1', pwdegree='1000', phdegree='1000', port='7687')

In [ ]:
%%time
# get orthopheno paths from a more open query topogology
seed = list([
        'HGNC:17646',  # NGLY1 human gene
        'HGNC:633'  # AQP1 human gene
])
hypothesis.open_query(seed,queryname='ngly1_aqp1',port='7687')

In [4]:
%%time
# get summary
data = summary.path_load('./hypothesis/query_ngly1_aqp1_pwdl1000_phdl1000_paths_v2020-09-14.json')

# parse data for summarization
data_parsed = list()
for query in data:
    query_parsed = summary.query_parser(query)
    data_parsed.append(query_parsed)
summary.metapaths(data_parsed)
summary.nodes(data_parsed)
summary.node_types(data_parsed)
summary.edges(data_parsed)
summary.edge_types(data_parsed)


The function "query_parser()" is running...

Finished query_parser().


The function "query_parser()" is running...

Finished query_parser().


The function "metapaths()" is running...

Printing summaries...


File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:17646_target:HGNC:633_summary_metapaths_v2020-09-21.csv' saved.

File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:17646_target:HGNC:633_summary_entities_in_metapaths_v2020-09-21.csv' saved.

Printing summaries...


File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:633_target:HGNC:17646_summary_metapaths_v2020-09-21.csv' saved.

File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:633_target:HGNC:17646_summary_entities_in_metapaths_v2020-09-21.csv' saved.

Finished metapaths().


The function "nod